# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выберите данные по странам своей группы (совместно): <br>
    3530203_80101: Spain, Portugal, France, Italy, Belgium<br>
    3530203_80102: Germany, Netherlands <br>
    3530903_80301: Belarus, Ukraine, Poland, Russian Federation<br>
    3530903_80302: Sweden, Finland, Norway, Denmark, Iceland<br>
    
2. Попытайтесь найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Найдите и объясните хотя бы 5 правил.
3. Выведите эти правила в отдельных ячейках. 
4. Подумайте, как можно было бы использовать полученные правила на практике.

In [23]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules, fpmax

In [24]:
data = pd.read_csv("lastfm.csv")
data.head(7)

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany
5,1,schandmaul,f,Germany
6,1,edguy,f,Germany


In [71]:
data_SPFIB=data[(data.country=="Spain")|(data.country=="Portugal")|(data.country=="France")|(data.country=="Italy")|(data.country=="Belgium")]
data_SPFIB['artist'].unique()

array(['lily allen', 'kanye west', 'sigur rós', 'pink floyd',
       'stevie wonder', 'metallica', 'thievery corporation',
       'iron maiden', 'the streets', 'muse', 'faith no more', 'manu chao',
       'tenacious d', 'depeche mode', 'justin timberlake', 'green day',
       'snow patrol', 'dream theater', 'u2', 'jay-z', 'type o negative',
       'pearl jam', 'queen', 'the beatles', 'lynyrd skynyrd',
       'neil young', 'the rolling stones', 'the who', 'led zeppelin',
       'black sabbath', 'the police', 't. rex', 'simon & garfunkel',
       'eagles', 'jimi hendrix', 'jethro tull', 'bee gees', 'van halen',
       'bob dylan', 'kiss', 'ac/dc', 'aerosmith', 'ennio morricone',
       'scorpions', 'bruce springsteen', 'dire straits', 'guns n roses',
       'inxs', 'rammstein', 'm.i.a.', 'fugazi', 'johnny cash',
       'clint mansell', 'elton john', 'portishead', 'social distortion',
       'rise against', 'pj harvey', 'radiohead', 'zero 7',
       'einstürzende neubauten', 'david bowie'

In [46]:
data_SPFIB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27214 entries, 83 to 289762
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user     27214 non-null  int64 
 1   artist   27214 non-null  object
 2   sex      27214 non-null  object
 3   country  27214 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.0+ MB


### Data preprocessing

In [26]:
# dummy encoding for artist
piv_artist = data_SPFIB[['user', 'artist']].pivot_table(index='user', columns='artist',aggfunc=any, fill_value=False).astype(int)
piv_artist.sum()

artist
...and you will know us by the trail of dead     8
2pac                                            16
3 doors down                                    29
30 seconds to mars                              42
311                                              3
                                                ..
yeah yeah yeahs                                 26
yellowcard                                       8
yo la tengo                                     11
zero 7                                          13
Édith piaf                                      17
Length: 994, dtype: int64

In [27]:
# dummy encoding for country
piv_country = data_SPFIB[['user', 'country']].pivot_table(index='user', columns='country',aggfunc=any, fill_value=False).astype(int)
piv_country.sum()

country
Belgium     160
France      327
Italy       304
Portugal    122
Spain       506
dtype: int64

In [28]:
# dummy encoding for sex
piv_sex = data_SPFIB[['user', 'sex']].pivot_table(index='user', columns='sex',aggfunc=any, fill_value=False).astype(int)
piv_sex.sum()

sex
f     312
m    1107
dtype: int64

In [7]:
piv_all = pd.concat([piv_artist, piv_country, piv_sex], axis=1)
piv_all.head()

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,[unknown],...,yo la tengo,zero 7,Édith piaf,Belgium,France,Italy,Portugal,Spain,f,m
user,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
36,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
47,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
74,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [6]:
piv_artist_country = pd.concat([piv_artist, piv_country], axis=1)
piv_artist_country.head()

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,[unknown],...,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf,Belgium,France,Italy,Portugal,Spain
user,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
36,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
47,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
74,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [31]:
piv_artist_sex = pd.concat([piv_artist, piv_sex], axis=1)
piv_artist_sex.head()

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,[unknown],...,wolfgang amadeus mozart,wu-tang clan,yann tiersen,yeah yeah yeahs,yellowcard,yo la tengo,zero 7,Édith piaf,f,m
user,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
36,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
47,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
74,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [32]:
piv_all = pd.concat([piv_artist, piv_sex, piv_country], axis=1)
piv_all.head()

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,[unknown],...,yo la tengo,zero 7,Édith piaf,f,m,Belgium,France,Italy,Portugal,Spain
user,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
36,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
47,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0
74,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


# Rule 1

In [8]:
apriori_artist = apriori(piv_artist, min_support=0.03, use_colnames=True)
apriori_artist

,support,itemsets
0,0.041579,([unknown])
1,0.043693,(a perfect circle)
2,0.073996,(ac/dc)
3,0.036646,(aerosmith)
4,0.092319,(air)
...,...,...
264,0.035941,"(radiohead, u2)"
265,0.035941,"(the rolling stones, the beatles)"
266,0.031712,"(the beatles, the strokes)"
267,0.039464,"(the killers, the strokes)"


In [9]:
apriori_artist_rules = association_rules(apriori_artist, metric="confidence", min_threshold=0.2)
apriori_artist_rules.sort_values('confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
40,(keane),(coldplay),0.038055,0.188161,0.033122,0.870370,4.625676,0.025961,6.262760
19,(beck),(radiohead),0.047216,0.237491,0.032417,0.686567,2.890916,0.021204,2.432766
32,(blur),(radiohead),0.047921,0.237491,0.030303,0.632353,2.662637,0.018922,2.074024
56,(snow patrol),(coldplay),0.050740,0.188161,0.031712,0.625000,3.321629,0.022165,2.164905
91,(kaiser chiefs),(the killers),0.050740,0.109937,0.031008,0.611111,5.558761,0.025430,2.288735
...,...,...,...,...,...,...,...,...,...
3,(radiohead),(air),0.237491,0.092319,0.048626,0.204748,2.217840,0.026701,1.141376
132,(the beatles),(nirvana),0.174066,0.124736,0.035236,0.202429,1.622864,0.013524,1.097413
68,(the beatles),(david bowie),0.174066,0.088090,0.035236,0.202429,2.297976,0.019903,1.143359
61,(coldplay),(the strokes),0.188161,0.087385,0.038055,0.202247,2.314426,0.021612,1.143982


Метрика lift — это отношение «зависимости» items к их «независимости». Lift показывает, насколько items зависят друг от друга. В случае, если lift = 1, мы говорим, что items независимы и правил совместного прослушивания тут нет. Если же lift>1, то величина, на которую lift, собственно, больше этой самой единицы, и покажет нам «силу» правила. Чем больше единицы, тем чаще посылка и следствие встречаются в транзакциях вместе.

In [10]:
rule1 = apriori_artist_rules[(apriori_artist_rules['antecedents'] == {'the beatles'})].sort_values(by = ['lift'],ascending=False)
rule1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
159,(the beatles),(the rolling stones),0.174066,0.072586,0.035941,0.206478,2.844582,0.023306,1.168730
68,(the beatles),(david bowie),0.174066,0.088090,0.035236,0.202429,2.297976,0.019903,1.143359
95,(the beatles),(led zeppelin),0.174066,0.098661,0.039464,0.226721,2.297976,0.022291,1.165606
140,(the beatles),(pink floyd),0.174066,0.130374,0.048626,0.279352,2.142707,0.025932,1.206729
149,(the beatles),(radiohead),0.174066,0.237491,0.070472,0.404858,1.704730,0.029133,1.281222
132,(the beatles),(nirvana),0.174066,0.124736,0.035236,0.202429,1.622864,0.013524,1.097413
57,(the beatles),(coldplay),0.174066,0.188161,0.047216,0.271255,1.441614,0.014464,1.114024
119,(the beatles),(muse),0.174066,0.174066,0.036646,0.210526,1.209461,0.006346,1.046183


# Rule 2

**fp-growth** позволяет установить меньший порог и получить больше наборов, среди которых могут быть интересные правила

In [26]:
fpgrowth_artist = fpgrowth(piv_artist, min_support=0.003, use_colnames=True)
len(fpgrowth_artist)

96631

In [44]:
fpgrowth_artist_rules = association_rules(fpgrowth_artist, metric="confidence", min_threshold=0.1)
fpgrowth_artist_rules.sort_values('support', ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
885928,(coldplay),(radiohead),0.188161,0.237491,0.083157,0.441948,1.860901,0.038471,1.366375
885929,(radiohead),(coldplay),0.237491,0.188161,0.083157,0.350148,1.860901,0.038471,1.249269
0,(muse),(radiohead),0.174066,0.237491,0.080338,0.461538,1.943392,0.038999,1.416088
1,(radiohead),(muse),0.237491,0.174066,0.080338,0.338279,1.943392,0.038999,1.248160
130498,(the beatles),(radiohead),0.174066,0.237491,0.070472,0.404858,1.704730,0.029133,1.281222
130499,(radiohead),(the beatles),0.237491,0.174066,0.070472,0.296736,1.704730,0.029133,1.174429
2,(muse),(coldplay),0.174066,0.188161,0.068358,0.392713,2.087113,0.035606,1.336829
3,(coldplay),(muse),0.188161,0.174066,0.068358,0.363296,2.087113,0.035606,1.297202
340881,(coldplay),(the killers),0.188161,0.109937,0.055673,0.295880,2.691371,0.034987,1.264079
340880,(the killers),(coldplay),0.109937,0.188161,0.055673,0.506410,2.691371,0.034987,1.644765


In [43]:
rule2 = fpgrowth_artist_rules[fpgrowth_artist_rules['lift'] < 1]
rule2 = rule2[(rule2['antecedents'].str.contains('radiohead', regex=False))]
rule2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
112,(radiohead),(metallica),0.237491,0.123326,0.024665,0.103858,0.842136,-0.004624,0.978275
138,"(the beatles, radiohead)",(metallica),0.070472,0.123326,0.007752,0.110000,0.891943,-0.000939,0.985027
156103,"(radiohead, air)",(the killers),0.048626,0.109937,0.004933,0.101449,0.922798,-0.000413,0.990554
156900,"(radiohead, david bowie)",(metallica),0.035236,0.123326,0.003524,0.100000,0.810857,-0.000822,0.974082
158004,"(radiohead, björk)",(red hot chili peppers),0.040874,0.103594,0.004228,0.103448,0.998593,-0.000006,0.999837
199927,"(nine inch nails, radiohead)",(coldplay),0.030303,0.188161,0.005638,0.186047,0.988764,-0.000064,0.997403
328003,"(the white stripes, radiohead)",(metallica),0.033122,0.123326,0.003524,0.106383,0.862614,-0.000561,0.981040
332476,"(beirut, radiohead)",(pink floyd),0.028189,0.130374,0.003524,0.125000,0.958784,-0.000151,0.993859
341848,"(radiohead, the cure)",(metallica),0.046512,0.123326,0.004933,0.106061,0.860000,-0.000803,0.980686
593692,"(beck, radiohead)",(the killers),0.032417,0.109937,0.003524,0.108696,0.988712,-0.000040,0.998608


Значение метрики lift < 1 говорит о том, что человек вряд ли будет слушать consequents, если слушает antecedents.
Также можно обратить внимание на метрику сonviction. В общем виде Conviction — это «частотность ошибок» нашего правила. Т.е. например, как часто слушали radiohead без metallica и наоборот.

# Rule 3

In [61]:
fpgrowth_country = fpgrowth(piv_artist_country, min_support=0.01, use_colnames=True)

In [62]:
rules_fpgrowth_country = association_rules(fpgrowth_country, metric="lift", min_threshold=0.4)
rules_fpgrowth_country.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Spain),(muse),0.356589,0.174066,0.067653,0.189723,1.089949,0.005583,1.019323
1,(muse),(Spain),0.174066,0.356589,0.067653,0.388664,1.089949,0.005583,1.052467
2,(France),(muse),0.230444,0.174066,0.038760,0.168196,0.966274,-0.001353,0.992942
3,(muse),(France),0.174066,0.230444,0.038760,0.222672,0.966274,-0.001353,0.990002
4,(radiohead),(muse),0.237491,0.174066,0.080338,0.338279,1.943392,0.038999,1.248160
5,(muse),(radiohead),0.174066,0.237491,0.080338,0.461538,1.943392,0.038999,1.416088
6,(coldplay),(muse),0.188161,0.174066,0.068358,0.363296,2.087113,0.035606,1.297202
7,(muse),(coldplay),0.174066,0.188161,0.068358,0.392713,2.087113,0.035606,1.336829
8,(muse),(Italy),0.174066,0.214235,0.033827,0.194332,0.907096,-0.003465,0.975296
9,(Italy),(muse),0.214235,0.174066,0.033827,0.157895,0.907096,-0.003465,0.980796


In [84]:
rules_fpgrowth_country["consequents_len"] = rules_fpgrowth_country["consequents"].apply(lambda x: len(x))
rule3 = rules_fpgrowth_country[rules_fpgrowth_country['lift'] < 1]
rule3 = rule3[((rules_fpgrowth_country['antecedents'] == {'France'}))
& (rule3['consequents'].map(lambda x: (('France' not in x))))
& (rule3['consequents_len'] == 1)]
rule3

C:\Users\E784~1\AppData\Local\Temp/ipykernel_16248/3410073240.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rule4 = rule4[((rules_fpgrowth_country['antecedents'] == {'France'}))


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,consequents_len
3,(France),(muse),0.230444,0.174066,0.038760,0.168196,0.966274,-0.001353,0.992942,1
289,(France),(depeche mode),0.230444,0.096547,0.019732,0.085627,0.886895,-0.002516,0.988057,1
374,(France),(u2),0.230444,0.088090,0.017618,0.076453,0.867890,-0.002682,0.987399,1
586,(France),(sigur rós),0.230444,0.086681,0.016913,0.073394,0.846722,-0.003062,0.985661,1
728,(France),(iron maiden),0.230444,0.073291,0.013390,0.058104,0.792784,-0.003500,0.983876,1
1169,(France),(the beatles),0.230444,0.174066,0.037350,0.162080,0.931137,-0.002762,0.985695,1
1418,(France),(bob dylan),0.230444,0.074700,0.010571,0.045872,0.614073,-0.006643,0.969785,1
1482,(France),(ac/dc),0.230444,0.073996,0.014094,0.061162,0.826562,-0.002957,0.986330,1
1545,(France),(the rolling stones),0.230444,0.072586,0.014094,0.061162,0.842612,-0.002633,0.987832,1
2580,(France),(the strokes),0.230444,0.087385,0.015504,0.067278,0.769902,-0.004634,0.978442,1


Consequents содержит артистов, которые не очень популярны среди пользователей во Франции (lift < 1). Поэтому предлагать клиентам слушать композиции этих исполнителей не стоит.

### Rule 4

In [19]:
fpgrowth_sex = fpgrowth(piv_artist_sex, min_support=0.05, use_colnames=True)

In [20]:
rules_fpgrowth_sex = association_rules(fpgrowth_sex, metric="lift", min_threshold=0.8)
rules_fpgrowth_sex.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(muse),(m),0.174066,0.780127,0.128964,0.740891,0.949705,-0.006830,0.848573
1,(m),(muse),0.780127,0.174066,0.128964,0.165312,0.949705,-0.006830,0.989512
2,(radiohead),(muse),0.237491,0.174066,0.080338,0.338279,1.943392,0.038999,1.248160
3,(muse),(radiohead),0.174066,0.237491,0.080338,0.461538,1.943392,0.038999,1.416088
4,(coldplay),(muse),0.188161,0.174066,0.068358,0.363296,2.087113,0.035606,1.297202
5,(muse),(coldplay),0.174066,0.188161,0.068358,0.392713,2.087113,0.035606,1.336829
6,"(muse, radiohead)",(m),0.080338,0.780127,0.064834,0.807018,1.034470,0.002160,1.139343
7,"(radiohead, m)",(muse),0.193094,0.174066,0.064834,0.335766,1.928958,0.031223,1.243439
8,"(muse, m)",(radiohead),0.128964,0.237491,0.064834,0.502732,2.116846,0.034207,1.533397
9,(radiohead),"(muse, m)",0.237491,0.128964,0.064834,0.272997,2.116846,0.034207,1.198119


In [21]:
rules_fpgrowth_sex["consequents_len"] = rules_fpgrowth_sex["consequents"].apply(lambda x: len(x))
rule4 = rules_fpgrowth_sex[rules_fpgrowth_sex['lift'] > 1]
rule4 = rule4[((rules_fpgrowth_sex['antecedents'] == {'m'}))
& (rule4['consequents'].map(lambda x: (('m' not in x))))
& (rule4['consequents_len'] == 1)]
rule4

C:\Users\E784~1\AppData\Local\Temp/ipykernel_7480/906572802.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  rule4 = rule4[((rules_fpgrowth_sex['antecedents'] == {'m'}))


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,consequents_len
19,(m),(pink floyd),0.780127,0.130374,0.112051,0.143631,1.101692,0.010343,1.015482,1
23,(m),(metallica),0.780127,0.123326,0.109232,0.140018,1.135346,0.013022,1.019409,1
27,(m),(u2),0.780127,0.088090,0.077519,0.099368,1.128022,0.008798,1.012522,1
28,(m),(queen),0.780127,0.086681,0.069767,0.089431,1.031727,0.002145,1.003020,1
31,(m),(sigur rós),0.780127,0.086681,0.069767,0.089431,1.031727,0.002145,1.003020,1
33,(m),(iron maiden),0.780127,0.073291,0.063425,0.081301,1.109287,0.006249,1.008719,1
35,(m),(pearl jam),0.780127,0.069063,0.059197,0.075881,1.098722,0.005319,1.007378,1
36,(m),(the beatles),0.780127,0.174066,0.137421,0.176152,1.011981,0.001627,1.002531,1
47,(m),(led zeppelin),0.780127,0.098661,0.083157,0.106594,1.080410,0.006189,1.008880,1
49,(m),(bob dylan),0.780127,0.074700,0.061311,0.078591,1.052079,0.003035,1.004222,1


In [63]:
fpgrowth_country = fpgrowth(piv_artist_country, min_support=0.01, use_colnames=True)

In [73]:
rules_fpgrowth_country = association_rules(fpgrowth_country, metric="lift", min_threshold=1)
rules_fpgrowth_country

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Spain),(muse),0.356589,0.174066,0.067653,0.189723,1.089949,0.005583,1.019323
1,(muse),(Spain),0.174066,0.356589,0.067653,0.388664,1.089949,0.005583,1.052467
2,(radiohead),(muse),0.237491,0.174066,0.080338,0.338279,1.943392,0.038999,1.248160
3,(muse),(radiohead),0.174066,0.237491,0.080338,0.461538,1.943392,0.038999,1.416088
4,(coldplay),(muse),0.188161,0.174066,0.068358,0.363296,2.087113,0.035606,1.297202
...,...,...,...,...,...,...,...,...,...
16167,"(coldplay, oasis)",(stereophonics),0.047921,0.021142,0.010571,0.220588,10.433824,0.009558,1.255894
16168,"(stereophonics, oasis)",(coldplay),0.011980,0.188161,0.010571,0.882353,4.689359,0.008317,6.900634
16169,(coldplay),"(stereophonics, oasis)",0.188161,0.011980,0.010571,0.056180,4.689359,0.008317,1.046830
16170,(stereophonics),"(coldplay, oasis)",0.021142,0.047921,0.010571,0.500000,10.433824,0.009558,1.904158


In [75]:
rule5 = rules_fpgrowth_country[
    ((rules_fpgrowth_country['consequents'] == {'Spain'})
      | (rules_fpgrowth_country['consequents'] == {'Portugal'})
      | (rules_fpgrowth_country['consequents'] == {'France'})
      | (rules_fpgrowth_country['consequents'] == {'Italy'})
      | (rules_fpgrowth_country['consequents'] == {'Belgium'}))
    & (rules_fpgrowth_country['antecedents'] == {'sigur rós'})].sort_values(by='lift', ascending=False)
rule5

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
647,(sigur rós),(Portugal),0.086681,0.085976,0.010571,0.121951,1.418433,0.003118,1.040972
542,(sigur rós),(Italy),0.086681,0.214235,0.024665,0.284553,1.328225,0.006095,1.098285
573,(sigur rós),(Belgium),0.086681,0.112755,0.011980,0.138211,1.225762,0.002207,1.029538
